In [1]:
import sys
import time
import joblib
import logging
import warnings
import requests
import swifter
import multiprocessing
import pandas as pd
import numpy as np
import yfinance as yf
import pysentiment as ps

from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from zero_shot_learner import extend_df_with_cos_sim
from preprocessor import NewsPreprocessor
from preprocessor import transform_df
from contractions import contractions_dict
from summarizer import Summarizer
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger()
logger.disabled = True

In [2]:
def add_content(url, semaphore, my_queue, ratio=0.8):
    """
    Return:
        res_origin: complete paragraph string
        res_ps: important sentence string
        res_bertsum: filtered string by BertSum
    """
    semaphore.acquire()
    #print(url+" : start")
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
    resp = requests.get(url, timeout=10, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')
    paragraph = soup.find_all('p')
    paragraph = [p.text for p in paragraph]
    paragraph = paragraph[1:-1]
    res_origin = "".join(paragraph)
#     hiv4_function = ps.hiv4.HIV4()
#     po = []
#     for p in paragraph:
#         tokens = hiv4_function.tokenize(p)
#         s = hiv4_function.get_score(tokens)
#         po.append(s['Polarity'])
#     res = []
#     for i, p in enumerate(po):
#         if(float(p) >= 0.85 or float(p) <= -0.85):
#             res.append(paragraph[i])
#     res_ps = "".join(res)
#     bert_summarizer = Summarizer()
#     result = bert_summarizer(res_origin, ratio=ratio)
#     res_bertsum = ''.join(result)
    # df = pd.Series([url, res_origin], columns=["url", "content"])
    my_queue.put([url, res_origin])
    while my_queue.qsize() % 50 == 0:
        print(my_queue.qsize())
    #print(url+" : end")
    semaphore.release()
    #return res_origin

In [3]:
# start = time.time()
# df = joblib.load("../data/sp500_top100_v1.bin")
# df.drop_duplicates(subset="title", inplace=True)
# df = df.iloc[0:10000, :]
# print("Get content...")
# tqdm.pandas()
# # df[["content"]] = df.progress_apply(lambda row: pd.Series(add_content(row["url"])), axis=1)
# # df[["content"]] = df.swifter.allow_dask_on_strings().apply(lambda row: pd.Series(add_content(row["url"])), axis=1)
# df["content"] = df["url"].swifter.allow_dask_on_strings().apply(add_content, axis=1)
# print("Done!")
# print(time.time()-start)
# df

In [4]:
# import multiprocessing
# pbar = tqdm()

# def process_df_function(df):
#     df["content"] = df["url"].swifter.allow_dask_on_strings().apply(add_content, axis=1)
#     pbar.update(1)
#     return df

# # load raw data into a dataframe
# raw_df = joblib.load("../data/sp500_top100_v1.bin")
# raw_df.drop_duplicates(subset="title", inplace=True)
# NUM_CORES = 8

# # split the raw dataframe into chunks
# df_chunks = np.array_split(raw_df, NUM_CORES)

# # use a pool to spawn multiple proecsses
# with multiprocessing.Pool(NUM_CORES) as pool:
#     # concatenate all processed chunks together.
#     # process_df_function is the function you defined in the previous block
#     print("Start")
#     processed_df = pd.concat(pool.map(process_df_function, df_chunks), ignore_index=True)
#     print("End")

In [5]:
import sys
def progressbar(iter, prefix="", size=60, file=sys.stdout):
    # Reference from https://stackoverflow.com/questions/3160699/python-progress-bar
    count = len(iter)
    def show(t):
        x = int(size*t/count)
        # file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), int(100*t/count), 100))
        file.write("{}[{}{}] {}%\r".format(prefix, "#"*x, "."*(size-x), int(100*t/count)))
        file.flush()
    show(0)
    for i, item in enumerate(iter):
        yield item
        show(i+1)
    file.write("\n")
    file.flush()

In [ ]:
import threading
import queue

df = joblib.load("../data/sp500_top100_v1.bin")
df.drop_duplicates(subset="title", inplace=True)
df.drop_duplicates(subset="url", inplace=True)
df = df.iloc[0:10000, :]

threads = []
semaphore = threading.Semaphore(10)
my_queue = queue.Queue()

for index, row in df.iterrows():
    threads.append(threading.Thread(target=add_content, args=(row["url"], semaphore, my_queue)))
    #row["content"] = add_content(row["url"])

for thread in threads:
    thread.start()
for thread in tqdm(threads):
    thread.join()

# print(my_queue.qsize())
temp_list = []
while my_queue.qsize() > 0:
    # print(my_queue.get())
    temp_list.append(my_queue.get())
df_temp = pd.DataFrame(temp_list, columns=["url", "content"])
df = pd.merge(df, df_temp, on="url")
df

50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
1

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300


300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300


400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400
400


...

1900
1900
1900
1900
1900
1900
1900
1900
1900
1900
1900
1900
1900
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
1950
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000


2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000


2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300
2300


2400
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450
2450


2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2600
2650
2650
2650
2650
2650
2650
2650
2650
2650
2650
2650
2650
2650
2650
2650
2650
2700
2700
2700
2700
2700
2700
2700
2700
2700
2700
2700
2750
2750
2750
2750
2800
2800
2800
2800
2800
2800
2800
2800
2800
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2850
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
2900
